# Gloo Gateway TAP Filter

https://docs.solo.io/gateway/1.19.x/

### Install Istio

In [ ]:
istioctl install  --skip-confirmation

### Install GG 1.19

In [ ]:
kubectl apply -f https://github.com/kubernetes-sigs/gateway-api/releases/download/v1.2.1/standard-install.yaml
helm repo add glooe https://storage.googleapis.com/gloo-ee-helm
helm repo update

In [ ]:
helm install -n gloo-system gloo glooe/gloo-ee \
--create-namespace \
--version 1.19.10 \
--set-string license_key=$GLOO_GATEWAY_LICENSE_KEY \
-f - <<EOF
gloo:
  discovery:
    enabled: false
  gatewayProxies:
    gatewayProxy:
      disabled: true
  kubeGateway:
    enabled: true
  gloo:
    disableLeaderElection: true
gloo-fed:
  enabled: false
  glooFedApiserver:
    enable: false
grafana:
  defaultInstallationEnabled: false
observability:
  enabled: false
prometheus:
  enabled: false
EOF

In [ ]:
kubectl apply -n gloo-system -f- <<EOF
kind: Gateway
apiVersion: gateway.networking.k8s.io/v1
metadata:
  name: http
spec:
  gatewayClassName: gloo-gateway
  listeners:
  - protocol: HTTP
    port: 8080
    name: http
    allowedRoutes:
      namespaces:
        from: All
EOF

### Deploy httpbin sample

In [ ]:
kubectl create ns httpbin
kubectl label namespace httpbin istio-injection=enabled
kubectl -n httpbin apply -f https://raw.githubusercontent.com/solo-io/gloo-mesh-use-cases/main/policy-demo/httpbin.yaml
sleep 4
kubectl get pods -n httpbin

### Create a route for httpbin

In [ ]:
kubectl apply -f- <<EOF
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: httpbin
  namespace: httpbin
  labels:
    example: httpbin-route
spec:
  parentRefs:
    - name: http
      namespace: gloo-system
  rules:
    - backendRefs:
        - name: httpbin
          port: 8000
EOF

### Send traffic to httpbin from GG

In [ ]:
export INGRESS_GW_ADDRESS=$(kubectl get svc -n gloo-system gloo-proxy-http -o=jsonpath="{.status.loadBalancer.ingress[0]['hostname','ip']}")
curl -i http://$INGRESS_GW_ADDRESS:8080/headers

## Capturing Gateway Traffic

### Deploy TAP Server

In [ ]:
kubectl apply -f ./tap-server.yaml

### Configure GG with the TAP Server

In [ ]:
kubectl apply -f- <<EOF
apiVersion: gateway.solo.io/v1
kind: HttpListenerOption
metadata:
  name: tapping
  namespace: gloo-system
spec:
  targetRefs:
  - group: gateway.networking.k8s.io
    kind: Gateway
    name: http
  options:
    tap:
      sinks:
      - httpService:
          tapServer:
            name: sample-tap-server-http
            namespace: gloo-system
          timeout: '20s'
EOF

### Send Traffic and Check TAP Server

In [ ]:
export INGRESS_GW_ADDRESS=$(kubectl get svc -n gloo-system gloo-proxy-http -o=jsonpath="{.status.loadBalancer.ingress[0]['hostname','ip']}")
curl -i http://$INGRESS_GW_ADDRESS:8080/headers

In [ ]:
kubectl -n gloo-system logs deployments/sample-tap-server-http --tail=100

## Capturing Mesh Traffic

Capture sidecar mesh traffic, by creating a VS to route thru the GG Gateway

### Deploy Sleep Sample

In [54]:
kubectl apply -f https://raw.githubusercontent.com/istio/istio/refs/heads/master/samples/sleep/sleep.yaml -n httpbin
sleep 4
kubectl get pods -n httpbin

serviceaccount/sleep created
service/sleep created
deployment.apps/sleep created
NAME                       READY   STATUS            RESTARTS   AGE
httpbin-789cf5c944-9tcm5   4/4     Running           0          8m22s
sleep-7964877df7-zftzx     1/2     PodInitializing   0          4s


In [56]:
kubectl exec deploy/sleep -n httpbin -- curl http://httpbin:8000/headers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{
  "headers": {
    "Accept": [
      "*/*"
    ],
    "Host": [
      "httpbin:8000"
    ],
    "User-Agent": [
      "curl/8.16.0"
    ],
    "X-Envoy-Attempt-Count": [
      "1"
    ],
    "X-Forwarded-Client-Cert": [
      "By=spiffe://cluster.local/ns/httpbin/sa/httpbin;Hash=ac9a015a1dd5ee6c3991fb262f0951c038ab373b885ff1de5d6cb68794b540f3;Subject=\"\";URI=spiffe://cluster.local/ns/httpbin/sa/sleep"
    ],
    "X-Forwarded-Proto": [
      "http"
    ],
    "X-Request-Id": [
      "33e0232c-85fe-424e-89b6-d92524ec1f8e"
    ]
  }
}
100   541  100   541    0     0  68299      0 --:--:-- --:--:-- --:--:-- 77285


In [57]:
kubectl apply -f- <<EOF
apiVersion: networking.istio.io/v1
kind: VirtualService
metadata:
  name: httpbin-hairpin-mesh
  namespace: httpbin
spec:
  hosts:
  - httpbin.httpbin.svc.cluster.local
  gateways:
  - mesh
  http:
  - route:
    - destination:
        host: gloo-proxy-http.gloo-system.svc.cluster.local
        port:
          number: 8080
      headers:
        request:
          set:
            Host: httpbin.httpbin.svc.cluster.local
EOF


virtualservice.networking.istio.io/httpbin-hairpin-mesh created


In [58]:
kubectl exec deploy/sleep -n httpbin -- curl http://httpbin:8000/headers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{
  "headers": {
    "Accept": [
      "*/*"
    ],
    "Host": [
      "httpbin.httpbin.svc.cluster.local"
    ],
    "User-Agent": [
      "curl/8.16.0"
    ],
    "X-Envoy-Attempt-Count": [
      "1"
    ],
    "X-Envoy-Original-Host": [
      "httpbin:8000"
    ],
    "X-Forwarded-Proto": [
      "http"
    ],
    "X-Request-Id": [
      "12e2e62c-cfb2-45e4-8ee9-a149228028cf"
    ]
  }
}
100   395  100   395    0     0  32443      0 --:--:-- --:--:-- --:--:-- 32916


In [ ]:
kubectl -n gloo-system logs deployments/sample-tap-server-http --tail=100

## Clean up

In [ ]:
kubectl delete -f ./tap-server.yaml
kubectl delete ns httpbin